In [ ]:
import numpy as np
import numpy.linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import data_receiver
from mathlib import *
from plotlib import *

In [ ]:
%matplotlib widget
np.set_printoptions(precision=8, suppress=True)

In [ ]:
# sampling rate
dt = 0.01    # s

# the initialization interval
ts = 1    # s

# pull data from phone
data order: gyroscorpe, accelerometer, magnetometer

In [ ]:
r = data_receiver.Receiver()

data = []

for line in r.receive():
    data.append(line.split(','))

data = np.array(data, dtype = np.float)

## Initialization

In [ ]:
# discard the first and last few readings
# for some reason they fluctuate a lot
w = data[10:-10, 0:3]
a = data[10:-10, 3:6]
m = data[10:-10, 6:9]

if(np.shape(w)[0] < ts/dt):
    print("not enough data for intialization!")

# gravity
gn = a[:int(ts/dt)].mean(axis = 0)
gn = -gn[:, np.newaxis]
g0 = np.linalg.norm(gn)  # save the initial magnitude of gravity

# magnetic field
mn = m[:int(ts/dt)].mean(axis = 0)
mn = Normalized(mn)[:, np.newaxis]  # magnitude is not important

# compute error covariance
avar = a[:int(ts/dt)].var(axis=0)
mvar = m[:int(ts/dt)].var(axis=0)
wvar = w[:int(ts/dt)].var(axis=0)

# cut the initialization data
w = w[int(ts/dt) - 1:] - w[:int(ts/dt)].mean(axis=0)
a = a[int(ts/dt):]
m = m[int(ts/dt):]

In [ ]:
a_filtered, w_filtered = Filt_signal((a, w), dt=dt, wn=10)
plot_signal(a, a_filtered, w, w_filtered, m)

In [ ]:
print("acc variance: ", avar)
print("mag variance: ", mvar)
print("angular variance: ", wvar)

## Kalman Filter

In [ ]:
gyro_noise = 2e-3
acc_noise = 2e-2
mag_noise = 2e-3

P = 1e-8 * I(4)

In [ ]:
qs = []
orientation = []
a_body = []
g = [gn.T[0]]

# state vector
ori = -gn / np.linalg.norm(gn)

q = np.array([[1, 0, 0, 0]]).T
t = 0
while t < np.shape(a)[0]:
    wt = w_filtered[t, np.newaxis].T
    at = a_filtered[t, np.newaxis].T
    mt = m[t, np.newaxis].T 
    mt = Normalized(mt)

    # Propagation
    Ft = F(q, wt, dt)
    Gt = G(q)
    Q = (gyro_noise * dt)**2 * Gt @ Gt.T
    
    q = Ft @ q
    q = Normalized(q)
    P = Ft @ P @ Ft.T + Q    

    # Measurement Update
    # Use only normalized measurements to reduce error!
    
    # acc and mag prediction
    pa = Normalized(-Rotate(q) @ gn)
    pm = Normalized(Rotate(q) @ mn)

    # Residual
    Eps = np.vstack((Normalized(at), mt)) - np.vstack((pa, pm))
    
    # internal error + external error
    Ra = [(acc_noise / np.linalg.norm(at))**2 + (1 - g0 / np.linalg.norm(at))**2] * 3
    Rm = [mag_noise**2] * 3
    R = np.diag(Ra + Rm)
    
    Ht = H(q, gn, mn)

    S = Ht @ P @ Ht.T + R
    K = P @ Ht.T @ np.linalg.inv(S)
    q = q + K @ Eps
    P = P - K @ Ht @ P
    
    # Post Correction
    q = Normalized(q)
    P = 0.5 * (P + P.T)  # make sure P is symmertical
    
    gt = Rotate(q) @ gn
    g.append(gt.T[0])
    a_body.append((at + gt).T[0])
    
    t += 1

orientation = np.array(orientation)
g = np.array(g)
a_body = np.array(a_body)

## plotting results

In [ ]:
plot_3D([[g, "gravity"]])

In [ ]:
plot_g_and_acc(g, a_body)

In [ ]:
plt.close('all')